In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1264897,2021-05-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1264898,2021-05-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1264899,2021-05-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1264900,2021-05-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
33703,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33705,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33707,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33709,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33711,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
777596,2021-05-11,Arkansas,Arkansas,5001,2105,36.00,5000,Arkansas,AR,Arkansas,State,3017804
777598,2021-05-12,Arkansas,Arkansas,5001,2109,36.00,5000,Arkansas,AR,Arkansas,State,3017804
777600,2021-05-13,Arkansas,Arkansas,5001,2110,36.00,5000,Arkansas,AR,Arkansas,State,3017804
777602,2021-05-14,Arkansas,Arkansas,5001,2112,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1264897,2021-05-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1264898,2021-05-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1264899,2021-05-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1264900,2021-05-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-15') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
480,2021-05-15,Snohomish,Washington,37807,574.00,53061,WA,County,822083,4598.93,69.82
958,2021-05-15,Cook,Illinois,546360,10691.00,17031,IL,County,5150233,10608.45,207.58
1435,2021-05-15,Orange,California,271194,5030.00,6059,CA,County,3175692,8539.68,158.39
1911,2021-05-15,Maricopa,Arizona,544255,9950.00,4013,AZ,County,4485414,12133.89,221.83
2387,2021-05-15,Los Angeles,California,1237899,24101.00,6037,CA,County,10039107,12330.77,240.07
...,...,...,...,...,...,...,...,...,...,...,...
1264165,2021-05-15,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08
1264380,2021-05-15,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1264564,2021-05-15,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1264744,2021-05-15,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
480,2021-05-15,Snohomish,Washington,37807,574.00,53061,WA,County,822083,4598.93,69.82,69.82,4598.93
958,2021-05-15,Cook,Illinois,546360,10691.00,17031,IL,County,5150233,10608.45,207.58,207.58,10608.45
1435,2021-05-15,Orange,California,271194,5030.00,6059,CA,County,3175692,8539.68,158.39,158.39,8539.68
1911,2021-05-15,Maricopa,Arizona,544255,9950.00,4013,AZ,County,4485414,12133.89,221.83,221.83,12133.89
2387,2021-05-15,Los Angeles,California,1237899,24101.00,6037,CA,County,10039107,12330.77,240.07,240.07,12330.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264165,2021-05-15,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08,75.08,2402.40
1264380,2021-05-15,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1264564,2021-05-15,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1264744,2021-05-15,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
813119,2021-05-15,Hale,Alabama,2239,78.00,1065,AL,County,14651,15282.23,532.39,532.39,15282.23,1
885139,2021-05-15,Clarke,Alabama,3514,61.00,1025,AL,County,23622,14875.96,258.23,258.23,14875.96,2
636081,2021-05-15,Lowndes,Alabama,1405,53.00,1085,AL,County,9726,14445.82,544.93,544.93,14445.82,3
563946,2021-05-15,Etowah,Alabama,13918,353.00,1055,AL,County,102268,13609.34,345.17,345.17,13609.34,4
473166,2021-05-15,Franklin,Alabama,4262,82.00,1059,AL,County,31362,13589.69,261.46,261.46,13589.69,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198419,2021-05-15,Platte,Wyoming,630,11.00,56031,WY,County,8393,7506.26,131.06,131.06,7506.26,19
775951,2021-05-15,Converse,Wyoming,1003,17.00,56009,WY,County,13822,7256.55,122.99,122.99,7256.55,20
964896,2021-05-15,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21
1074062,2021-05-15,Niobrara,Wyoming,161,2.00,56027,WY,County,2356,6833.62,84.89,84.89,6833.62,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
636081,2021-05-15,Lowndes,Alabama,1405,53.00,1085,AL,County,9726,14445.82,544.93,544.93,14445.82,3,1
813119,2021-05-15,Hale,Alabama,2239,78.00,1065,AL,County,14651,15282.23,532.39,532.39,15282.23,1,2
270385,2021-05-15,Walker,Alabama,7260,278.00,1127,AL,County,63521,11429.29,437.65,437.65,11429.29,28,3
562278,2021-05-15,Clay,Alabama,1571,57.00,1027,AL,County,13235,11870.04,430.68,430.68,11870.04,20,4
685998,2021-05-15,Greene,Alabama,926,34.00,1063,AL,County,8111,11416.59,419.18,419.18,11416.59,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684337,2021-05-15,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
940824,2021-05-15,Uinta,Wyoming,2239,13.00,56041,WY,County,20226,11069.91,64.27,64.27,11069.91,5,20
964896,2021-05-15,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21,21
269116,2021-05-15,Teton,Wyoming,3765,9.00,56039,WY,County,23464,16045.86,38.36,38.36,16045.86,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4598.93,19,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4598.93,19,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4598.93,19,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4598.93,19,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4598.93,19,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261936,2021-05-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1261937,2021-05-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1261938,2021-05-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1261939,2021-05-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
809747,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15282.23,2,1,1.00
809748,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15282.23,2,1,0.00
809749,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15282.23,2,1,0.00
809750,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15282.23,2,1,0.00
809751,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15282.23,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981060,2021-05-11,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,1.00
981061,2021-05-12,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,0.00
981062,2021-05-13,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,0.00
981063,2021-05-14,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
633533,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14445.82,1,3,1.00,0.00
633534,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14445.82,1,3,0.00,0.00
633535,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14445.82,1,3,0.00,0.00
633536,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14445.82,1,3,0.00,0.00
633537,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14445.82,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631448,2021-05-11,Albany,Wyoming,4373,12.00,56001,WY,County,38880,11247.43,30.86,30.86,11350.31,23,4,8.00,0.00
631449,2021-05-12,Albany,Wyoming,4388,12.00,56001,WY,County,38880,11286.01,30.86,30.86,11350.31,23,4,15.00,0.00
631450,2021-05-13,Albany,Wyoming,4406,12.00,56001,WY,County,38880,11332.30,30.86,30.86,11350.31,23,4,18.00,0.00
631451,2021-05-14,Albany,Wyoming,4413,12.00,56001,WY,County,38880,11350.31,30.86,30.86,11350.31,23,4,7.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-15') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
322805,2021-05-15,Imperial,California,28485,727.00,6025,CA,County,181215,15718.90,401.18,401.18,15718.90,1,2,17.00,-1.00
2387,2021-05-15,Los Angeles,California,1237899,24101.00,6037,CA,County,10039107,12330.77,240.07,240.07,12330.77,2,5,247.00,18.00
160677,2021-05-15,San Bernardino,California,297165,4721.00,6071,CA,County,2180085,13630.89,216.55,216.55,13630.89,3,4,262.00,-1.00
565566,2021-05-15,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
71821,2021-05-15,Stanislaus,California,62249,1061.00,6099,CA,County,550660,11304.43,192.68,192.68,11304.43,5,9,61.00,0.00
33267,2021-05-15,Riverside,California,299793,4594.00,6065,CA,County,2470546,12134.69,185.95,185.95,12134.69,6,7,0.00,0.00
57135,2021-05-15,San Joaquin,California,73310,1394.00,6077,CA,County,762148,9618.87,182.90,182.90,9618.87,7,18,0.00,0.00
89058,2021-05-15,Tulare,California,49830,840.00,6107,CA,County,466195,10688.66,180.18,180.18,10688.66,8,10,0.00,0.00
32397,2021-05-15,Fresno,California,101917,1688.00,6019,CA,County,999101,10200.87,168.95,168.95,10200.87,9,14,55.00,1.00
512497,2021-05-15,Merced,California,31919,461.00,6047,CA,County,277680,11494.89,166.02,166.02,11494.89,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1197970,2021-05-15,Lassen,California,5706,24.00,6035,CA,County,30573,18663.53,78.50,78.50,18663.53,34,1,0.00,0.00
322805,2021-05-15,Imperial,California,28485,727.00,6025,CA,County,181215,15718.90,401.18,401.18,15718.90,1,2,17.00,-1.00
685940,2021-05-15,Kings,California,23013,246.00,6031,CA,County,152940,15047.08,160.85,160.85,15047.08,11,3,5.00,0.00
160677,2021-05-15,San Bernardino,California,297165,4721.00,6071,CA,County,2180085,13630.89,216.55,216.55,13630.89,3,4,262.00,-1.00
2387,2021-05-15,Los Angeles,California,1237899,24101.00,6037,CA,County,10039107,12330.77,240.07,240.07,12330.77,2,5,247.00,18.00
202041,2021-05-15,Kern,California,109272,1376.00,6029,CA,County,900202,12138.61,152.85,152.85,12138.61,14,6,86.00,2.00
33267,2021-05-15,Riverside,California,299793,4594.00,6065,CA,County,2470546,12134.69,185.95,185.95,12134.69,6,7,0.00,0.00
512497,2021-05-15,Merced,California,31919,461.00,6047,CA,County,277680,11494.89,166.02,166.02,11494.89,10,8,0.00,0.00
71821,2021-05-15,Stanislaus,California,62249,1061.00,6099,CA,County,550660,11304.43,192.68,192.68,11304.43,5,9,61.00,0.00
89058,2021-05-15,Tulare,California,49830,840.00,6107,CA,County,466195,10688.66,180.18,180.18,10688.66,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
322805,2021-05-15,Imperial,California,28485,727.00,6025,CA,County,181215,15718.90,401.18,401.18,15718.90,1,2,17.00,-1.00
2387,2021-05-15,Los Angeles,California,1237899,24101.00,6037,CA,County,10039107,12330.77,240.07,240.07,12330.77,2,5,247.00,18.00
160677,2021-05-15,San Bernardino,California,297165,4721.00,6071,CA,County,2180085,13630.89,216.55,216.55,13630.89,3,4,262.00,-1.00
565566,2021-05-15,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
71821,2021-05-15,Stanislaus,California,62249,1061.00,6099,CA,County,550660,11304.43,192.68,192.68,11304.43,5,9,61.00,0.00
33267,2021-05-15,Riverside,California,299793,4594.00,6065,CA,County,2470546,12134.69,185.95,185.95,12134.69,6,7,0.00,0.00
57135,2021-05-15,San Joaquin,California,73310,1394.00,6077,CA,County,762148,9618.87,182.90,182.90,9618.87,7,18,0.00,0.00
89058,2021-05-15,Tulare,California,49830,840.00,6107,CA,County,466195,10688.66,180.18,180.18,10688.66,8,10,0.00,0.00
32397,2021-05-15,Fresno,California,101917,1688.00,6019,CA,County,999101,10200.87,168.95,168.95,10200.87,9,14,55.00,1.00
512497,2021-05-15,Merced,California,31919,461.00,6047,CA,County,277680,11494.89,166.02,166.02,11494.89,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,401.18,15718.90,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4232,05/11/21,Lassen,5701,24.00,18647.17,78.50,78.50,18663.53,34,1,3.00,0.00
4233,05/12/21,Lassen,5701,24.00,18647.17,78.50,78.50,18663.53,34,1,0.00,0.00
4234,05/13/21,Lassen,5701,24.00,18647.17,78.50,78.50,18663.53,34,1,0.00,0.00
4235,05/14/21,Lassen,5706,24.00,18663.53,78.50,78.50,18663.53,34,1,5.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,401.18,15718.90,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,401.18,15718.90,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4318,05/11/21,Merced,31859,460.00,11473.28,165.66,166.02,11494.89,10,8,38.00,1.00
4319,05/12/21,Merced,31898,460.00,11487.32,165.66,166.02,11494.89,10,8,39.00,0.00
4320,05/13/21,Merced,31914,461.00,11493.09,166.02,166.02,11494.89,10,8,16.00,1.00
4321,05/14/21,Merced,31919,461.00,11494.89,166.02,166.02,11494.89,10,8,5.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)